In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import joblib
import random
import numpy as np


In [49]:
data = pd.read_csv('availability.csv')
data.rename(columns={'last_update': 'time_of_day'}, inplace=True)
data

,station_id,time_of_day,available_bike_stands,bike_stands,available_bikes,status
0,1,2024-02-21 22:18:52,9,31,22,OPEN
1,1,2024-02-21 22:37:12,8,31,23,OPEN
2,1,2024-02-21 22:47:17,8,31,23,OPEN
3,1,2024-02-21 22:57:38,7,31,24,OPEN
4,1,2024-02-21 23:00:01,8,31,23,OPEN
...,...,...,...,...,...,...
565009,117,2024-03-20 22:19:17,29,40,11,OPEN
565010,117,2024-03-20 22:23:42,29,40,11,OPEN
565011,117,2024-03-20 22:28:23,29,40,11,OPEN
565012,117,2024-03-20 22:38:28,29,40,11,OPEN


In [50]:

data2 = pd.read_csv('weather.csv')
data2

,time_of_day,station_id,main,description,temperature,wind_speed
0,2024-02-25 15:03:04,1,Clouds,broken clouds,8,9.77
1,2024-02-25 15:03:04,9,Clouds,broken clouds,8,9.77
2,2024-02-25 15:03:04,14,Clouds,broken clouds,8,9.77
3,2024-02-25 15:03:04,27,Clouds,broken clouds,8,9.77
4,2024-02-25 15:03:04,36,Clouds,broken clouds,8,9.77
...,...,...,...,...,...,...
198240,2024-03-20 22:40:19,106,Clear,clear sky,5,4.12
198241,2024-03-20 22:40:19,107,Clear,clear sky,6,4.12
198242,2024-03-20 22:40:20,108,Clear,clear sky,5,4.12
198243,2024-03-20 22:40:20,112,Clear,clear sky,6,4.12


In [56]:
import pandas as pd

# Load your two dataframes
df1 = data
df2 = data2
# Convert 'time_of_day' column to datetime format
df1['time_of_day'] = pd.to_datetime(df1['time_of_day'])
df2['time_of_day'] = pd.to_datetime(df2['time_of_day'])
# Sort dataframes by 'time_of_day'
df1.sort_values('time_of_day', inplace=True)
df2.sort_values('time_of_day', inplace=True)

# Merge dataframes using merge_asof
merged_df = pd.merge_asof(df1, df2, by='station_id', on='time_of_day', direction="nearest")




In [57]:
merged_df.head(1500)

,station_id,time_of_day,available_bike_stands,bike_stands,available_bikes,status,time_of_day_seconds_x,main,description,temperature,wind_speed,time_of_day_seconds_y
0,72,2024-02-21 22:09:43,31,31,0,OPEN,79783,Clouds,broken clouds,8,9.77,54298
1,73,2024-02-21 22:09:46,20,30,10,OPEN,79786,Clouds,broken clouds,8,9.77,54298
2,26,2024-02-21 22:16:21,20,20,0,OPEN,80181,Clouds,broken clouds,8,9.77,54246
3,54,2024-02-21 22:18:11,33,33,0,OPEN,80291,Clouds,broken clouds,8,9.77,54184
4,16,2024-02-21 22:18:20,1,20,19,OPEN,80300,Clouds,broken clouds,8,9.77,54290
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,67,2024-02-22 00:16:36,30,40,10,OPEN,996,Clouds,broken clouds,8,9.77,54302
1496,111,2024-02-22 00:16:38,33,40,7,OPEN,998,Clouds,broken clouds,7,9.77,54292
1497,53,2024-02-22 00:16:40,40,40,0,OPEN,1000,Clouds,broken clouds,8,9.77,54184
1498,62,2024-02-22 00:16:40,32,40,8,OPEN,1000,Clouds,broken clouds,8,9.77,54566


In [64]:
data = merged_df

data['month'] = pd.to_datetime(data['time_of_day']).dt.month
data['day'] = pd.to_datetime(data['time_of_day']).dt.day
data['hour'] = pd.to_datetime(data['time_of_day']).dt.hour
data['minute'] = pd.to_datetime(data['time_of_day']).dt.minute
data = pd.get_dummies(data, columns=['main', "description"], prefix='category')

data = data.drop(columns=["status", "time_of_day"])
X = data.drop('available_bikes', axis=1)  # Features
Y = data['available_bikes']  # Target variable

X


,station_id,available_bike_stands,bike_stands,time_of_day_seconds_x,temperature,wind_speed,time_of_day_seconds_y,month,day,hour,...,category_light intensity drizzle,category_light intensity drizzle rain,category_light intensity shower rain,category_light rain,category_mist,category_moderate rain,category_overcast clouds,category_scattered clouds,category_sleet,category_snow
0,72,31,31,79783,8,9.77,54298,2,21,22,...,False,False,False,False,False,False,False,False,False,False
1,73,20,30,79786,8,9.77,54298,2,21,22,...,False,False,False,False,False,False,False,False,False,False
2,26,20,20,80181,8,9.77,54246,2,21,22,...,False,False,False,False,False,False,False,False,False,False
3,54,33,33,80291,8,9.77,54184,2,21,22,...,False,False,False,False,False,False,False,False,False,False
4,16,1,20,80300,8,9.77,54290,2,21,22,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565009,28,26,30,82169,6,4.12,81605,3,20,22,...,False,False,False,False,False,False,False,False,False,False
565010,108,27,35,82173,5,4.12,81620,3,20,22,...,False,False,False,False,False,False,False,False,False,False
565011,56,27,40,82174,6,4.12,81611,3,20,22,...,False,False,False,False,False,False,False,False,False,False
565012,90,37,40,82177,6,4.12,81617,3,20,22,...,False,False,False,False,False,False,False,False,False,False


In [65]:
Y

0          0
1         10
2          0
3          0
4         19
          ..
565009     4
565010     8
565011    13
565012     3
565013     0
Name: available_bikes, Length: 565014, dtype: int64

In [66]:
# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=52)


In [67]:
# Initialize Naive Bayes classifier
model = GaussianNB()

# Train the model
model.fit(X_train, y_train)


GaussianNB()

In [68]:
y_pred = model.predict(X_test)

In [69]:
from sklearn import linear_model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 54.57006451156164
R-squared: 0.4552974089078907


In [70]:


reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [71]:
reg.coef_

array([ 1.15065268e-03, -9.85006690e-01,  9.87618834e-01,  3.27890459e-05,
       -9.48942011e-03, -4.35821291e-03,  2.09434079e-07, -3.26097813e-01,
       -1.53664641e-02, -1.16655981e-01, -1.99758000e-03,  3.87001054e-02,
        5.52889749e-02,  1.00442077e-01, -2.72089244e-01,  5.80836832e-02,
        8.16049244e-02, -6.20305206e-02,  4.69587657e-02,  3.87001054e-02,
       -2.45350312e-02, -2.72089244e-01, -5.85976601e-02, -1.52493708e-02,
        1.15691447e-01,  5.81580161e-02,  4.57348337e-02,  5.80836832e-02,
        3.63097348e-02,  4.98140833e-02, -1.69488429e-02, -3.10875284e-02,
       -3.09429921e-02])

In [72]:
# Predictions on the testing set
y_pred = model.predict(X_test)


In [73]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 54.57006451156164
R-squared: 0.4552974089078907


In [74]:
# Predictions on the testing set
y_pred = model.predict(X_test)


In [75]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.14237675105970637
